In [2]:
import numpy as np
import os
import socket
from DataAPI import *
os.getcwd()

'E:\\researches\\EQTest\\SocialAffordance'

###  Open Maya server: run the follow command in the script editor in Maya

### open server
import maya.cmds as cmds

cmds.commandPort(n="localhost:12345")

### close server

cmds.commandPort(cl=True, n="localhost:12345")

In [ ]:
#controller = MayaController(PORT=12345)
controllee = MayaController(PORT=12346)

In [ ]:
skeleton = controllee.SendCommand("listRelatives -c -ad Character1_Hips")[:-2].split("\t")
skeleton.append('Character1_Hips')
skeleton_ori = ["mixamorig:" + s[11:] for s in skeleton]
len(skeleton_ori)

In [ ]:
skl = 'Character1_Hips' # Skeleton name
face = 'woman2_geo_rig_copy:face' # face name

In [ ]:
def Preparation():
    # Hide unnecessary models
    controllee.SendCommand('hide woman2_geo_rig_copy:body;')
    controllee.SendCommand('hide woman2_geo_rig_copy:mixamorig:FitSkeleton;')
    controllee.SendCommand('hide woman2_geo_rig_copy:left;')
    controllee.SendCommand('parent -w woman2_geo_rig_copy:hair;')
    controllee.SendCommand('hide woman2_geo_rig_copy:hair;')
    controllee.SendCommand('parent -w woman2_geo_rig_copy:eye_L;')
    controllee.SendCommand('hide woman2_geo_rig_copy:eye_L;')
    controllee.SendCommand('parent -w woman2_geo_rig_copy:eye_R;')
    controllee.SendCommand('hide woman2_geo_rig_copy:eye_R;')

    controllee.SendCommand('jointDisplayScale -a 7.5;') # Set Joint size
    controllee.SendCommand('setAttr -lock off %s.translateX;' % face) # Unlock X
    controllee.SendCommand('setAttr -lock off %s.translateY;' % face) # Unlock Y
    controllee.SendCommand('setAttr -lock off %s.scaleX;' % face) # Unlock Scale X
    controllee.SendCommand('setAttr -lock off %s.scaleY;' % face) # Unlock Scale Y
    controllee.SendCommand('setAttr -lock off %s.scaleZ;' % face) # Unlock Scale Z
    controllee.SendCommand('scale -a 13 13 13 %s;' % face) # Set face scale

In [ ]:
def BF_ListToJson(data:list):
    return {"woman2_geo_rig_copy:Morpher": 
        {"Blink": data[0]
        ,"EyelidUp": data[1]
        ,"A": data[2]
        ,"E": data[3]
        ,"O": data[4]
        ,"MouthOpen": data[5]
        ,"FV": data[6]
        ,"M": data[7]
        ,"MouthUp": data[8]
        ,"MouthWide": data[9]
        ,"MouthWide2": data[10]
        ,"MouthDown": data[11]
        ,"MouthOneSideUp": data[12]
        ,"EyebrowInDown": data[13]
        ,"EyebrowOutDown": data[14]
        ,"EyebrowUp": data[15]
        ,"EyebrowSwag": data[16]
        }
    }
    # Usage:
    # pred = controllee.SetMultipleAttributes(json) (TODO)
    # controllee.SetMultipleAttributes(BF_ListToJson(pred))

In [ ]:
def FormSkeleton(total_frame, dist_frame):
    for i in range(0, total_frame, dist_frame):
        controller.SetCurrentTimeFrame(i)
        controllee.SetCurrentTimeFrame(i)
        for i in range(len(skeleton)):
            rot = controller.GetObjectLocalRoation(skeleton_ori[i])
            controllee.SetObjectLocalRotation(skeleton[i], rot, isRelative="")
        controllee.SetCurrentKeyFrameForObjects(skeleton)

    for i in range(0, total_frame, dist_frame):
        controllee.SetCurrentTimeFrame(i)
        skl_name = 'skl_temp' + str(i)
        controllee.SendCommand('duplicate -n %s %s;' % (skl_name, skl)) # Duplicate skeleton
        controllee.SendCommand('color -rgb 0 0 %f %s' % (i / 400, skl_name)) # Set color
        controllee.SendCommand('move -absolute %d 0 0 %s;' % (i / dist_frame * 15, skl_name)) # Move created skeleton
    
    controllee.SendCommand('move -absolute %d 0 0 %s;' % (total_frame / dist_frame * 15, skl))  # Set last display frame
    controllee.SetCurrentKeyFrameForPositionAndRotation(skl)
    controllee.SendCommand('color -rgb 0 0 %f %s' % (total_frame / 400, skl))

def FormFaces(total_frame, dist_frame):
    for i in range(0, total_frame, dist_frame):
        controllee.SetCurrentTimeFrame(i)
        face_name = 'face_temp' + str(i)
        controllee.SendCommand('duplicate -n %s %s;' % (face_name, face)) # Duplicate face 
        controllee.SendCommand('move -absolute %d 18 0 %s;' % (i / dist_frame * 15, face_name)) # move face
    
    controllee.SendCommand('move -absolute %d 18 0 %s;' % (total_frame / dist_frame * 15, face)) # move face


def Reset(total_frame, dist_frame):
    for i in range(0, total_frame, dist_frame):
        controllee.SendCommand('delete skl_temp%d;' % i)
        controllee.SendCommand('delete face_temp%d;' % i)

#### Usage:
Open 2 Maya windows (PORT = 12345 & 12346)

12345: 
         
         import Target file 

12346:   
         
         Open pure skeleton.mb

         import woman2_geo_rig_copy.mb
         
         import background.mb

In [ ]:
l = 40 # Total frame of the animation
d = 8 # distance of frames of 2 screenshots

Preparation()
FormFaces(l, d)
FormSkeleton(l, d)

In [ ]:
# Reset Scene
Reset(l, d)

In [ ]:
controller.Close()
controllee.Close()